In [5]:
%pip install numpy tensorflow scikit-learn

OK
  Using cached scipy-1.12.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (165 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 6.3 MB/s eta 0:00:00m eta 0:00:010:01010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 8.3 MB/s eta 0:00:00m eta 0:00:010:00:01m

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Specify each path

In [7]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [8]:
NUM_CLASSES = 7

# Dataset reading

In [9]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [10]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [13]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
Total params: 1158 (4.52 KB)
Trainable params: 1158 (4.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [15]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [16]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/32 [..............................] - ETA: 5s - loss: 2.2803 - accuracy: 0.0781
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
32/32 [==============================] - 0s 4ms/step - loss: 2.1654 - accuracy: 0.1391 - val_loss: 1.9852 - val_accuracy: 0.2907
Epoch 2/1000
 1/32 [..............................] - ETA: 0s - loss: 1.9895 - accuracy: 0.2578
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
32/32 [==============================] - 0s 1ms/step - loss: 1.9649 - accuracy: 0.2470 - val_loss: 1.8638 - val_accuracy: 0.2966
Epoch 3/1000
 1/32 [..............................] - ETA: 0s - loss: 1.9555 - accuracy: 0.2266
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
32/32 [==============================] - 0s 1ms/step - loss: 1.8653 - accuracy: 0.2739 - val_loss: 1.7729 - val_accuracy: 0.2981
Epoch 4/1000
 1/32 [..............................] - ETA: 0s - loss: 1.7586 - accuracy: 0.3125
E

/Users/chanakan5591/Developments/AI-hand-sign/env/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 6: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
32/32 [==============================] - 0s 1ms/step - loss: 1.6519 - accuracy: 0.2961 - val_loss: 1.5474 - val_accuracy: 0.3084
Epoch 7/1000
28/32 [=========================>....] - ETA: 0s - loss: 1.5783 - accuracy: 0.3398
Epoch 7: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
32/32 [==============================] - 0s 3ms/step - loss: 1.5775 - accuracy: 0.3415 - val_loss: 1.4635 - val_accuracy: 0.5007
Epoch 8/1000
 1/32 [..............................] - ETA: 0s - loss: 1.4894 - accuracy: 0.3750
Epoch 8: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
32/32 [==============================] - 0s 1ms/step - loss: 1.5174 - accuracy: 0.3937 - val_loss: 1.3930 - val_accuracy: 0.5496
Epoch 9/1000
 1/32 [..............................] - ETA: 0s - loss: 1.4260 - accuracy: 0.4453
Epoch 9: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
32/32 [=============

In [17]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

11/11 [==============================] - 0s 531us/step - loss: 0.3323 - accuracy: 0.9379


In [18]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [19]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 47ms/step
[1.9552467e-02 5.1604748e-02 8.9542466e-01 1.6133921e-05 1.7212700e-02
 2.4195931e-06 9.1160443e-03 7.0707574e-03]
2


# Confusion matrix

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

ModuleNotFoundError: No module named 'pandas'

# Convert to model for Tensorflow-Lite

In [21]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [22]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /var/folders/5b/rbzdlhp90pz1xbvnny45w40r0000gn/T/tmp3eycp9ap/assets


INFO:tensorflow:Assets written to: /var/folders/5b/rbzdlhp90pz1xbvnny45w40r0000gn/T/tmp3eycp9ap/assets
2024-02-14 06:15:20.734199: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-14 06:15:20.734304: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-14 06:15:20.734889: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/5b/rbzdlhp90pz1xbvnny45w40r0000gn/T/tmp3eycp9ap
2024-02-14 06:15:20.735531: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-14 06:15:20.735536: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/5b/rbzdlhp90pz1xbvnny45w40r0000gn/T/tmp3eycp9ap
2024-02-14 06:15:20.738512: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-02-14 06:15:20.739295: I tensorflow/cc/saved_model/load

6720

# Inference test

In [23]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [24]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [202]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [203]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

Wall time: 0 ns


In [204]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[7.0533440e-02 4.3624082e-01 4.6905449e-01 5.5653197e-03 3.3129596e-03
 6.9846800e-03 8.1667323e-03 1.4163637e-04]
2
